In [3]:
from pathlib import Path
import sys, os, requests, time
from datetime import datetime
from json import dumps, loads
from time import sleep
from random import randint
import numpy as np, pandas as pd
from kafka import KafkaConsumer, KafkaProducer
#
from data_util.stream.data_processing import processor, query_data
from data_util.stream.data_stream import kafka_streamer
%load_ext autoreload
%autoreload 2

In [4]:
class PATH:
    path = Path('./').absolute()
    data_path = path/'data'
    log_name = 'kafka_log-movielog6_stream.csv'
    rate_log_name = log_name.split('.')[0]+'-rateonly.csv'
    #log_output_name = 'kafka_log-movielog6(Full)-cleaned.csv'
    log_fp = data_path/log_name

In [5]:
# define parameters
topic = 'movielog6'

### Mongodb

In [16]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
conf = SparkConf().set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector:10.0.4")

import pymongo
from data_util.mongo_query import mongo_ops
mongo_local = mongo_ops(db='movielog', host = '127.0.0.1', port = 27017,)

In [22]:
# Pysparl not used
db = 'movielog'
collection = 'stream-rateonly'
mongo_spark = SparkSession \
    .builder \
    .appName("myApp") \
    .master("local") \
    .config('spark.driver.extraClassPath','/opt/spark/jars/*') \
    .config("spark.mongodb.input.uri", f"mongodb://127.0.0.1:27017/{db}/{collection}") \
    .config("spark.mongodb.output.uri", f"mongodb://127.0.0.1:27017/{db}/{collection}") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.4') \
    .getOrCreate()
#com.mongodb.spark.sql.connector.MongoTableProvider

<b> Mongo DB API for Join

In [55]:
conn = mongo_local.conn['user-rateonly']
cursor = conn.find().sort('timestamp').limit(2)
print(f"Collections:{mongo_local.conn.list_collection_names()}")
pd.DataFrame(list(cursor)).head(2)

Collections:['stream-rateonly', 'combined-rateonly', 'user-rateonly', 'movie-rateonly', 'stream', 'user-rateonly_nodup']


,_id,user_id,age,occupation,gender
0,6331f6adf6b96b5628d01783,244058,27,self-employed,M
1,6331f6adf6b96b5628d0179e,212777,31,self-employed,M


In [57]:
def join_by_id(db, stream_collection:str="streamCollection", user_collection:str="myUserCollection", movie_collection:str="myMovieCollection"):
    db[user_collection].aggregate(
        pipeline = [
            { "$sort": { "_id": 1 } }, 
            { "$group": { 
                "_id": "$user_id", 
                "doc": { "$first": "$$ROOT" } 
            }}, 
            { "$replaceRoot": { "newRoot": "$doc" } },
            { "$out": "user-rateonly_nodup" },
        ]
    )
    db[stream_collection].aggregate(
        pipeline = [
            {
                "$lookup": {
                    'from': 'movie-rateonly_nodub',
                    'localField': 'movie_id',
                    'foreignField': 'movie_id',
                    'as': 'movie_info'
                }
            },
            {
                "$lookup": {
                    'from': 'user-rateonly_nodub',
                    'localField': 'user_id',
                    'foreignField': 'user_id',
                    'as': 'user_info'
                }
            },
            { "$out": "combined-rateonly" }
        ]
    )
    return result
mongo_cursor = join_by_id(mongo_local.conn, user_collection='user-rateonly')

In [13]:
coll_name = ['movie-rateonly','stream-rateonly'][1]
movie_df = mongo_local.read(coll_name).sort_values('timestamp')
movie_df.head(5)

,_id,timestamp,user_id,request_type,movie_id,movie_time,rating
117798,6332170af6b96b5628d975e7,2022-07-29T21:35:11,189095,rate,the+ladies+man+2000,,2.0
16671,63321708f6b96b5628d7e953,2022-07-29T21:39,76270,rate,peter+pan+1953,,2.0
73403,63321709f6b96b5628d8c95b,2022-07-29T21:40:39,86226,rate,sixteen+candles+1984,,4.0
87658,6332170af6b96b5628d9031a,2022-07-29T21:43:26,196894,rate,21+2008,,5.0
74270,63321709f6b96b5628d8cf45,2022-07-29T21:49:52,91785,rate,flipper+1996,,4.0


<b> Mongo Atlas

In [6]:
atlas_mongo = mongo_ops(db='movielog',host='cluster0.xuddjop.mongodb.net', 
                        username='mlproduction', password='pulp_prediction', atlas=True)

In [19]:
movie_df.head(2)

,adult,genres,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,vote_average,vote_count,message
0,False,"[28, 12, 53]",en,2.826282,"[3146, 8411]",[United States of America],0,96.0,[English],Released,5.8,4,NaN
1,False,"[14, 12, 28, 10751, 10749]",en,19.467404,"[2, 3036, 16314]",[United States of America],758539785,97.0,[English],Released,7.0,4607,NaN


In [8]:
#atlas_mongo.import_df('stream-rateonly', df)
#atlas_mongo.import_df('user-rateonly', user_df)
movie_df = atlas_mongo.read('movie-rateonly')
print(movie_df.shape)
movie_df.head(2)

(124208, 13)


,adult,genres,original_language,popularity,production_companies,production_countries,revenue,runtime,spoken_languages,status,vote_average,vote_count,message
0,False,"[28, 12, 53]",en,2.826282,"[3146, 8411]",[United States of America],0,96.0,[English],Released,5.8,4,NaN
1,False,"[14, 12, 28, 10751, 10749]",en,19.467404,"[2, 3036, 16314]",[United States of America],758539785,97.0,[English],Released,7.0,4607,NaN
